In [21]:
import os
import json
from azure.ai.projects import AIProjectClient 
from azure.ai.projects.models._patch import FileSearchTool 
from azure.identity import DefaultAzureCredential
from IPython.display import Markdown, display
 

project_client = AIProjectClient.from_connection_string( 
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"] 
) 

 
with project_client: 
    openai_file = project_client.agents.upload_file_and_poll(file_path="../../data/product_info_1.md", purpose="assistants") 
    print(f"Uploaded file, file ID: {openai_file.id}") 
 
    openai_vectorstore = project_client.agents.create_vector_store_and_poll(file_ids=[openai_file.id], name="my_vectorstore") 
    print(f"Created vector store, vector store ID: {openai_vectorstore.id}") 
 
    # Create file search tool with resources 
    file_search = FileSearchTool(vector_store_ids=[openai_vectorstore.id]) 
 
    # Create agent with file search tool and process the agent run 
    agent = project_client.agents.create_agent( 
        model="gpt-4o-mini", 
        name="my-agent", 
        instructions="Hello, you are helpful agent and can search information from uploaded files", 
        tools=file_search.definitions, 
        tool_resources=file_search.resources, 
    ) 
    print(f"Created agent, agent ID: {agent.id}") 
 
    # Create thread for communication 
    thread = project_client.agents.create_thread() 
    print(f"Created thread, ID: {thread.id}") 
 
    # Create message to thread 
    message = project_client.agents.create_message( 
        thread_id=thread.id, role="user", content="Hello, what Contoso products do you know?" 
    ) 
    print(f"Created message, ID: {message.id}") 
 
    # Create and process agent run in thread with tools 
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id) 
    print(f"Run finished with status: {run.status}") 
 
    if run.status == "failed": 
        # Check if you got "Rate limit is exceeded.", then you want to get more quota 
        print(f"Run failed: {run.last_error}") 
 
    # Delete the file when done 
    project_client.agents.delete_vector_store(openai_vectorstore.id) 
    print("Deleted vector store") 
 
    # Delete the agent when done 
    project_client.agents.delete_agent(agent.id) 
    print("Deleted agent") 
 
    # Fetch and log all messages 
    messages = project_client.agents.list_messages(thread_id=thread.id) 
    print(f"Messages: {messages}")

    # Convert messages.data list into a dictionary
    messages_json = json.dumps(messages.data[0].as_dict())

    messages_dict = messages.data[0]

    # Print the value
    display(Markdown(messages_dict.content[0].text['value']))

    # Print formatted json
    print(json.dumps(json.loads(messages_json), indent=4, sort_keys=True))

Uploaded file, file ID: assistant-hZBWRXRWka4QA4TN8NMr3LrC
Created vector store, vector store ID: vs_1rjVkJDHykyUeGRMB0Pw1Yhs
Created agent, agent ID: asst_nu2UzO9ekrwzrrF82s789gPO
Created thread, ID: thread_5pR2JQbcABP3RjSGIeX0TAc4
Created message, ID: msg_c5cHJnrp4qBGnJEzi8b0TVVh
Run finished with status: RunStatus.COMPLETED
Deleted vector store
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_BJb2ZAUbOojeOboDEh8HEnMF', 'object': 'thread.message', 'created_at': 1731537826, 'assistant_id': 'asst_nu2UzO9ekrwzrrF82s789gPO', 'thread_id': 'thread_5pR2JQbcABP3RjSGIeX0TAc4', 'run_id': 'run_d9AwSfNdg4WKhah5QfwSA7Ei', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The document contains information about a specific product offered by Contoso: the **TrailMaster X4 Tent**. Here are the details:\n\n### TrailMaster X4 Tent\n- **Price**: $250\n- **Brand**: OutdoorLiving\n- **Category**: Tents\n- **Features**:\n  - Made from durable polyester material\n  - Spacio

The document contains information about a specific product offered by Contoso: the **TrailMaster X4 Tent**. Here are the details:

### TrailMaster X4 Tent
- **Price**: $250
- **Brand**: OutdoorLiving
- **Category**: Tents
- **Features**:
  - Made from durable polyester material
  - Spacious enough for up to 4 people
  - Easy setup with included instructions
  - Water-resistant and includes a rainfly for weather protection
  - Equipped with mesh panels for ventilation and insect protection
  - Comes with multiple doors, interior pockets, and a carry bag for storage.

### Technical Specifications
- **Best Use**: Camping
- **Capacity**: 4-person
- **Setup Style**: Freestanding
- **Weight**: 12 lbs
- **Dimensions**: 10ft x 8ft (with a peak height of 6ft)

### Additional Information
The tent also features warnings for care and maintenance, including avoiding sharp objects, not using harsh chemicals, and storing it dry to prevent mold. It comes with a 2-year limited warranty covering manufacturing defects and issues arising from normal use【4:0†source】.

If you are interested in more specific aspects about Contoso products or others, let me know!

{
    "assistant_id": "asst_nu2UzO9ekrwzrrF82s789gPO",
    "attachments": [],
    "content": [
        {
            "text": {
                "annotations": [
                    {
                        "end_index": 1061,
                        "file_citation": {
                            "file_id": "assistant-hZBWRXRWka4QA4TN8NMr3LrC"
                        },
                        "start_index": 1049,
                        "text": "\u30104:0\u2020source\u3011",
                        "type": "file_citation"
                    },
                    {
                        "end_index": 1061,
                        "file_citation": {
                            "file_id": "assistant-hZBWRXRWka4QA4TN8NMr3LrC"
                        },
                        "start_index": 1049,
                        "text": "\u30104:0\u2020source\u3011",
                        "type": "file_citation"
                    }
                ],
                "value": "The document con